# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstoneexperiment'

experiment=Experiment(ws, experiment_name)

ds = Dataset.get_by_name(ws,'alcohol')
ds.to_pandas_dataframe()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FR8A3KGMZ to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='Walc',
    n_cross_validations=2)

In [4]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [6]:
best_model, fitted_model = remote_run.get_output()
#print(best_model)
best_model.get_tags()

{'ensembled_iterations': '[33, 24, 48, 52, 16, 0, 19]',
 'ensembled_algorithms': "['ExtremeRandomTrees', 'ExtremeRandomTrees', 'ExtremeRandomTrees', 'XGBoostClassifier', 'ExtremeRandomTrees', 'LightGBM', 'ExtremeRandomTrees']",
 'ensemble_weights': '[0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]',
 'best_individual_pipeline_score': '0.5013331282366815',
 'best_individual_iteration': '33',
 'model_explanation': 'True'}

In [7]:
#TODO: Save the best model
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
#best_run = hdr.get_best_run_by_primary_metric()
#best_run_metrics = best_run.get_metrics()
parameter_values = best_model.get_details()

#print('Best Run Id: ', best_run.id)
#print('\n Accuracy:', best_run_metrics['Accuracy'])
#print('\n max iter:',parameter_values[3])
#print('\n C:',parameter_values[1])
#best_run.get_file_names()
os.makedirs('outputs',exist_ok=True)
filename = 'finalized_model.sav'
modeldoc = joblib.dump('outputs/model.joblib', filename)
print(modeldoc)
#model=best_model.register_model(model_name='automl_model', model_path= '/outputs/model.joblib')
#print(parameter_values)

['finalized_model.sav']


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [8]:
from azureml.core import Model

TODO: In the cell below, send a request to the web service you deployed to test it.

In [9]:
from azureml.core.model import Model
model=Model.register(ws,filename,'alcoholtest')

Registering model alcoholtest


TODO: In the cell below, print the logs of the web service and delete the service

In [47]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

from azureml.core.webservice import AciWebservice, Webservice

best_model.download_file('outputs/scoring_file_v_1_0_0.py','score.py')
env= best_model.download_file('outputs/conda_env_v_1_0_0.yml','myenv.yml')
#best_model.download_files('~/cloudfiles/code/Users/odl_user_137677/myenv.yml','myenv.yml')

#env=Environment.from_conda_specification(name='conda_env_v_1_0_0',file_path='outputs/conda_env_v_1_0_0.yml')
#env=Environment.from_conda_specification(file_path='outputs/conda_env_v_1_0_0.yml', name='myenv.yml')

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "capstonewebservice1",
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)


In [51]:
#service.wait_for_deployment(show_output = True)
print(service.scoring_uri)
print(service.swagger_uri)



None
None
